In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
import open3d as o3d
from rdkit import Chem
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
sys.path.extend([".", "../"])

In [4]:
from score_pcd import fit_and_score
from utils import align_and_show_scaffold

In [5]:
pcdref = o3d.io.read_point_cloud("ligands/BRD4/ligand-4_molgrid.pcd")
pcd = o3d.io.read_point_cloud("ligands/BRD4/ligand-4_murcko_tran_molgrid.pcd")

molref = next(Chem.SDMolSupplier("ligands/BRD4/ligand-4.sdf"))
mol = next(Chem.SDMolSupplier("ligands/BRD4/ligand-4_murcko_tran.sdf"))
scaffoldref = next(Chem.SDMolSupplier("ligands/BRD4/ligand-4_murcko.sdf"))

In [6]:
for _ in range(10):
    fit, cfit, tran = fit_and_score((pcd, pcdref), voxel_size=0.5, threshold=0.5)
    print(f"{cfit.inlier_rmse:.5f}")

0.30033
0.30479
0.33437
0.33323
0.30033
0.30033
0.30479
0.30008
0.30350
0.30008


In [7]:
for _ in range(10):
    ri, rf, _, _ = align_and_show_scaffold(molref, pcdref, mol, pcd, scaffoldref)
    print(f"{rf:.5f}")

0.25707
0.21341
0.25707
3.85652
0.16924
0.16924
0.21341
0.14095
0.16924
0.14095


In [9]:
from rdkit.Chem import AllChem

mol = Chem.MolFromSmiles("COO")
molh = Chem.AddHs(mol)
ids = AllChem.EmbedMultipleConfs(molh, numConfs=2)
list(ids)

print("molh(0)\n", molh.GetConformer(0).GetPositions()[0])
print("molh(1)\n", molh.GetConformer(1).GetPositions()[0])

# Copy original molecule without conformers
cmolh = Chem.Mol(molh, True)

conf = molh.GetConformer(0)
conf.SetId(0)
cmolh.AddConformer(conf, assignId=False)
print("cmol(0)\n", cmolh.GetConformer(0).GetPositions()[0])

conf = molh.GetConformer(1)
conf.SetId(0)  # Same ID; trying to overwrite...
# Need to explicitly add RemoveConformer() to remove the old conformer
cmolh.AddConformer(conf, assignId=False)
print("cmol(0)\n", cmolh.GetConformer(0).GetPositions()[0])

molh(0)
 [-0.70870826 -0.00343504  0.02374574]
molh(1)
 [-0.74559093 -0.04608781  0.04707924]
cmol(0)
 [-0.70870826 -0.00343504  0.02374574]
cmol(0)
 [-0.70870826 -0.00343504  0.02374574]
